In [52]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cadd_io as cdio
import cadd_main as cdmain
import mdutilities_io as mduio
import mymath as Mmath
import my_plot as myplot
import cadddatadump as cddump
import cadd_material as cdm
import cadd_mesh2 as cdmesh
import lineplot_oo3 as lpoo
import os
%matplotlib qt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preliminaries

## Notes

In [53]:
# If I use the values I think are correct (from Segurado and LLorca, 2007),
# there's a factor of two difference between my calculation and Srinath's, for some reason.
# On the other hand, I'm not positive he used the Segurado  values for b and the elastic constants
# If I assume that he specified Lobs incorrectly, and use Lobs = 400 nm, not 200 nm, I get the "right" answer.
# (See Figure 2 of Chakravarthy and Curtin, 2011, MSMSE)

## Directories

In [54]:
maindir = '../Tests/Source_with_Obstacles/'
uidir = maindir + 'User Inputs/'
fidir = maindir + 'Fortran Inputs/'
dumpdir = maindir + 'Dump Files/'
simname = 'source_with_obstacles'
simtype = 'dd'

## Unit System

In [55]:
# length - nanometers
# stress, elastic constants - MPa
# time - nanoseconds
# dislocation drag - MPa-ns
lengthfac = 1e9
stressfac = 1e-6
timefac = 1e9

## Misc

In [56]:
nfematerials = 1

# Create Inputs

## Nodes/Elements

In [57]:
bounds = np.array([-3,3,-3,3])*1e-6*lengthfac # 6 by 6 micron box
nvec = [4,4]
rmesh = cdmesh.MeshRectangle(bounds,nvec)

In [58]:
fixedbc = 3
box = cdmesh.Box(bounds)
rmeshnodes = rmesh.nodes
for line in box.lines:
    nodenums = rmeshnodes.search_for_nodes_along_line(line)
    rmeshnodes.set_node_bc(nodenums,fixedbc)

In [59]:
rmesh.write_user_inputs_all('feelementsmat1','nodes',subdir=uidir)

## Material

In [60]:
# test
mu = 10.608811378479004
nu = 0.25
E = 2*mu*(1+nu)
print(cdm.stiffness_2d_voigt(E,nu))

[[ 31.82643414  10.60881138   0.        ]
 [ 10.60881138  31.82643414   0.        ]
 [  0.           0.          10.60881138]]


In [61]:
E = 70e9*stressfac
nu = 0.33
mu = E/(2*(1+nu))
stiffnessmat = cdm.stiffness_2d_voigt(E,nu)
burgers = 0.25e-9*lengthfac
lannih = 6*burgers
B = 1e-4*stressfac*timefac
mname = 'Aluminum'
mass = 1.0 # fake
lattice = 'fcc' # fake
rho = 1.0 # fake
dislvmax = 1000*lengthfac/timefac

In [62]:
materialdict = {'burgers': burgers, 'elconst': stiffnessmat,
                'disldrag': B, 'dislvmax': dislvmax, 'lannih': lannih,
                'mname': mname, 'mass': mass,
                'lattice': lattice, 'rho': rho}
cdio.write_input(materialdict,uidir+'input.material1')

## DD Stuff

In [63]:
# timestep = 5e-10*timefac

### Sources

In [64]:
# need posn, slipsys, taucr, tnuc

In [65]:
# nucleation length
taucrsrc = 50e6*stressfac
tnuc = 0.01e-6*timefac # (20 timesteps)
lnuc = mu*burgers/(2*np.pi*(1-nu)*taucrsrc)

In [66]:
posnarray = np.zeros((1,2)).astype(float)
slipsysarray = np.ones((1,)).astype(int)
taucrarray = taucrsrc*np.ones((1,)).astype(float)
tnucarray = tnuc*np.ones((1,)).astype(float)

In [67]:
sourcedict = {'posn': posnarray, 'slipsys': slipsysarray, 'taucr': taucrarray, 'tnuc': tnucarray}
cdio.write_input(sourcedict,uidir+'input.sourcesmat1')

### Obstacles

In [68]:
# need posn, slipsys, taucr

In [69]:
taucrobs = 1e10*stressfac # essentially infinite strength
Lobs = 400e-9*lengthfac
posnarray = np.zeros((2,2)).astype(float)
# symmetric obstacles
posnarray[0,0] = -Lobs/2 
posnarray[1,0] = Lobs/2

slipsysarray = np.ones((2,)).astype(int)
taucrarray = taucrobs*np.ones((2,)).astype(float)

In [70]:
obstacledict = {'posn': posnarray, 'slipsys': slipsysarray, 'taucr': taucrarray}
cdio.write_input(obstacledict,uidir+'input.obstaclesmat1')

### Slip Systems

In [71]:
# need nslipplanes, origin, space, theta
nslipplanesarray = np.ones((1,)).astype(int)
originarray = np.zeros((1,2)).astype(float)
spacearray = np.ones((1,)).astype(float)
thetaarray = np.zeros((1,)).astype(float)


In [72]:
slipsysdict = {'nslipplanes': nslipplanesarray, 'origin': originarray, 'space': spacearray, 'theta': thetaarray}
cdio.write_input(slipsysdict,uidir+'input.slipsysmat1')

### Misc

In [73]:
# critical applied stress for first nucleation
# tausrc = tauapp + 2*taudisl
# where taudisl is given by vdG and Needleman expression
taucrsrc + 2*mu*burgers/(Lobs*np.pi*(1-nu))

65.62794020933772

# Write Inputs

In [74]:
# instantiate Simulation object, populate by reading from user inputs
cadddata = cdmain.Simulation(simtype,simname,uidir,fidir,dumpdir,nfematerials=nfematerials)

# write to fortran inputs
cadddata.write_fortran_all()

## Dump Files

In [75]:
axisbounds = np.array([-300,300,-300,300])
fig = cadddata.plot_dump_from_increment(140,axisbounds=axisbounds);

## Log Files

In [76]:
def plot_data(data,**kwargs):
    lineplotdata = lpoo.LinePlotData.from_dataset(data)
    myfig = myplot.my_plot(lineplotdata)
    myplot.pretty_figure(myfig,**kwargs)
    return myfig

def actual_predicted_data(data):
    stressapp = data[:,0]
    stresspinactual = data[:,1]
    ndisl = data[:,2]/2 # divide by 2 because symmetry
    stresspinpredict = stressapp*ndisl
    return np.column_stack((stressapp,stresspinactual)), np.column_stack((stressapp,stresspinpredict))   

def log_file_data(fac,vmax,gradientcorr):
    logfilepath = log_file_path(fac,vmax,gradientcorr)
    return np.loadtxt(logfilepath)

def log_file_path(fac,vmax,gradientcorr,logdir='Log Files/'):
    subdir = maindir + logdir
    suffix = get_suffix(gradientcorr)
    return '{subdir}{simname}_{fac}_vmax_{vmax}_{suffix}'.format(subdir=subdir,simname=simname,suffix=suffix,
                                                                 fac=fac,vmax=vmax)

def get_suffix(gradientcorr):
    if gradientcorr:
        return 'corr'
    else:
        return 'uncorr'


In [77]:
vmaxcorr = int(dislvmax)
data = log_file_data(1,vmaxcorr,True)
actualdata, predicteddata = actual_predicted_data(data)
dataall = [actualdata]
plot_data(dataall,axisbounds=[0,350,0,4500],xlabel='Applied shear stress (MPa)',ylabel='Pinning stress (MPa)');